<a href="https://colab.research.google.com/github/sartansartan/DS-Unit-2-Linear-Models/blob/master/module4-logistic-regression/ASartan_assignment_regression_classification_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 4*

---

# Logistic Regression


## Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.

- [ ] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
- [ ] Begin with baselines for classification.
- [ ] Use scikit-learn for logistic regression.
- [ ] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [ ] Get your model's test accuracy. (One time, at the end.)
- [ ] Commit your notebook to your fork of the GitHub repo.
- [ ] Watch Aaron's [video #1](https://www.youtube.com/watch?v=pREaWFli-5I) (12 minutes) & [video #2](https://www.youtube.com/watch?v=bDQgVt4hFgY) (9 minutes) to learn about the mathematics of Logistic Regression.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [ ] Do one-hot encoding.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
# Load data downloaded from https://srcole.github.io/100burritos/
import pandas as pd
df = pd.read_csv(DATA_PATH+'burritos/burritos.csv')

In [0]:
# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

In [0]:
# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

In [0]:
# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

In [0]:
# Drop some columns to prevent "leakage"
df = df.drop(columns=['Rec', 'overall'])

In [7]:
df.head()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
0,California,1/18/2016,3.5,4.2,NaN,6.49,3.0,NaN,NaN,NaN,NaN,NaN,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,California,1/24/2016,3.5,3.3,NaN,5.45,3.5,NaN,NaN,NaN,NaN,NaN,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,Carnitas,1/24/2016,NaN,NaN,NaN,4.85,1.5,NaN,NaN,NaN,NaN,NaN,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0,NaN,NaN,NaN,x,x,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,Asada,1/24/2016,NaN,NaN,NaN,5.25,2.0,NaN,NaN,NaN,NaN,NaN,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0,NaN,NaN,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,California,1/27/2016,4.0,3.8,x,6.59,4.0,NaN,NaN,NaN,NaN,NaN,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0,NaN,NaN,x,x,NaN,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [0]:
#Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.

In [9]:
df.shape

(421, 59)

In [10]:
df.isnull().sum()

Burrito             0
Date                0
Yelp              334
Google            334
Chips             395
Cost                7
Hunger              3
Mass (g)          399
Density (g/mL)    399
Length            138
Circum            140
Volume            140
Tortilla            0
Temp               20
Meat               14
Fillings            3
Meat:filling        9
Uniformity          2
Salsa              25
Synergy             2
Wrap                3
Unreliable        388
NonSD             414
Beef              242
Pico              263
Guac              267
Cheese            262
Fries             294
Sour cream        329
Pork              370
Chicken           400
Shrimp            400
Fish              415
Rice              385
Beans             386
Lettuce           410
Tomato            414
Bell peper        414
Carrots           420
Cabbage           413
Sauce             383
Salsa.1           414
Cilantro          406
Onion             404
Taquito           417
Pineapple 

In [0]:
thresh = len(df) * .2


In [12]:
thresh

84.2

In [0]:
df.dropna(thresh = thresh, axis = 1, inplace = True)

In [14]:
df.isnull().sum()

Burrito           0
Date              0
Yelp            334
Google          334
Cost              7
Hunger            3
Length          138
Circum          140
Volume          140
Tortilla          0
Temp             20
Meat             14
Fillings          3
Meat:filling      9
Uniformity        2
Salsa            25
Synergy           2
Wrap              3
Beef            242
Pico            263
Guac            267
Cheese          262
Fries           294
Sour cream      329
Great             0
dtype: int64

In [0]:
cols_mean = ['Yelp', 'Google', 'Length', 'Circum', 'Volume']

In [0]:
df['Yelp'].fillna((df['Yelp'].mean()), inplace=True)

In [0]:
df['Google'].fillna((df['Google'].mean()), inplace=True)

In [0]:
df['Length'].fillna((df['Length'].mean()), inplace=True)

In [0]:
df['Circum'].fillna((df['Circum'].mean()), inplace=True)

In [0]:
df['Volume'].fillna((df['Volume'].mean()), inplace=True)

In [0]:
cols = ['Beef', 'Pico', 'Guac', 'Cheese', 'Fries', 'Sour cream', 'Salsa']
df[cols] = df[cols].fillna(0)
#df[cols].fillna(0, inplace=True)

In [0]:
df[cols] = df[cols].replace(r'x', '1')

In [0]:
df = df.replace(r'X', '1')

In [24]:
df.head()

,Burrito,Date,Yelp,Google,Cost,Hunger,Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Beef,Pico,Guac,Cheese,Fries,Sour cream,Great
0,California,1/18/2016,3.500000,4.200000,6.49,3.0,20.038233,22.135765,0.786477,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,1,1,1,1,1,0,False
1,California,1/24/2016,3.500000,3.300000,5.45,3.5,20.038233,22.135765,0.786477,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,1,1,1,1,1,0,False
2,Carnitas,1/24/2016,3.887356,4.167816,4.85,1.5,20.038233,22.135765,0.786477,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0,0,1,1,0,0,0,False
3,Asada,1/24/2016,3.887356,4.167816,5.25,2.0,20.038233,22.135765,0.786477,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0,1,1,1,0,0,0,False
4,California,1/27/2016,4.000000,3.800000,6.59,4.0,20.038233,22.135765,0.786477,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0,1,1,0,1,1,0,True


In [0]:
df[cols] = df[cols].astype(int)

In [26]:
df.isnull().sum()

Burrito          0
Date             0
Yelp             0
Google           0
Cost             7
Hunger           3
Length           0
Circum           0
Volume           0
Tortilla         0
Temp            20
Meat            14
Fillings         3
Meat:filling     9
Uniformity       2
Salsa            0
Synergy          2
Wrap             3
Beef             0
Pico             0
Guac             0
Cheese           0
Fries            0
Sour cream       0
Great            0
dtype: int64

In [0]:
df.dropna(inplace = True)

In [28]:
df.head()

,Burrito,Date,Yelp,Google,Cost,Hunger,Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Beef,Pico,Guac,Cheese,Fries,Sour cream,Great
0,California,1/18/2016,3.500000,4.200000,6.49,3.0,20.038233,22.135765,0.786477,3.0,5.0,3.0,3.5,4.0,4.0,4,4.0,4.0,1,1,1,1,1,0,False
1,California,1/24/2016,3.500000,3.300000,5.45,3.5,20.038233,22.135765,0.786477,2.0,3.5,2.5,2.5,2.0,4.0,3,2.5,5.0,1,1,1,1,1,0,False
2,Carnitas,1/24/2016,3.887356,4.167816,4.85,1.5,20.038233,22.135765,0.786477,3.0,2.0,2.5,3.0,4.5,4.0,3,3.0,5.0,0,1,1,0,0,0,False
3,Asada,1/24/2016,3.887356,4.167816,5.25,2.0,20.038233,22.135765,0.786477,3.0,2.0,3.5,3.0,4.0,5.0,4,4.0,5.0,1,1,1,0,0,0,False
4,California,1/27/2016,4.000000,3.800000,6.59,4.0,20.038233,22.135765,0.786477,4.0,5.0,4.0,3.5,4.5,5.0,2,4.5,4.0,1,1,0,1,1,0,True


In [29]:
df.isnull().sum()

Burrito         0
Date            0
Yelp            0
Google          0
Cost            0
Hunger          0
Length          0
Circum          0
Volume          0
Tortilla        0
Temp            0
Meat            0
Fillings        0
Meat:filling    0
Uniformity      0
Salsa           0
Synergy         0
Wrap            0
Beef            0
Pico            0
Guac            0
Cheese          0
Fries           0
Sour cream      0
Great           0
dtype: int64

In [30]:
df['Date'] = pd.to_datetime(df['Date'])
df['Date'].dtypes

dtype('<M8[ns]')

In [31]:
df.head()

,Burrito,Date,Yelp,Google,Cost,Hunger,Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Beef,Pico,Guac,Cheese,Fries,Sour cream,Great
0,California,2016-01-18,3.500000,4.200000,6.49,3.0,20.038233,22.135765,0.786477,3.0,5.0,3.0,3.5,4.0,4.0,4,4.0,4.0,1,1,1,1,1,0,False
1,California,2016-01-24,3.500000,3.300000,5.45,3.5,20.038233,22.135765,0.786477,2.0,3.5,2.5,2.5,2.0,4.0,3,2.5,5.0,1,1,1,1,1,0,False
2,Carnitas,2016-01-24,3.887356,4.167816,4.85,1.5,20.038233,22.135765,0.786477,3.0,2.0,2.5,3.0,4.5,4.0,3,3.0,5.0,0,1,1,0,0,0,False
3,Asada,2016-01-24,3.887356,4.167816,5.25,2.0,20.038233,22.135765,0.786477,3.0,2.0,3.5,3.0,4.0,5.0,4,4.0,5.0,1,1,1,0,0,0,False
4,California,2016-01-27,4.000000,3.800000,6.59,4.0,20.038233,22.135765,0.786477,4.0,5.0,4.0,3.5,4.5,5.0,2,4.5,4.0,1,1,0,1,1,0,True


In [0]:
#Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.

In [33]:
mask = (df['Date'] < '2017-01-01')
train = df.loc[mask]
train.head()

,Burrito,Date,Yelp,Google,Cost,Hunger,Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Beef,Pico,Guac,Cheese,Fries,Sour cream,Great
0,California,2016-01-18,3.500000,4.200000,6.49,3.0,20.038233,22.135765,0.786477,3.0,5.0,3.0,3.5,4.0,4.0,4,4.0,4.0,1,1,1,1,1,0,False
1,California,2016-01-24,3.500000,3.300000,5.45,3.5,20.038233,22.135765,0.786477,2.0,3.5,2.5,2.5,2.0,4.0,3,2.5,5.0,1,1,1,1,1,0,False
2,Carnitas,2016-01-24,3.887356,4.167816,4.85,1.5,20.038233,22.135765,0.786477,3.0,2.0,2.5,3.0,4.5,4.0,3,3.0,5.0,0,1,1,0,0,0,False
3,Asada,2016-01-24,3.887356,4.167816,5.25,2.0,20.038233,22.135765,0.786477,3.0,2.0,3.5,3.0,4.0,5.0,4,4.0,5.0,1,1,1,0,0,0,False
4,California,2016-01-27,4.000000,3.800000,6.59,4.0,20.038233,22.135765,0.786477,4.0,5.0,4.0,3.5,4.5,5.0,2,4.5,4.0,1,1,0,1,1,0,True


In [34]:
mask = (df['Date'] > '2016-12-31') & (df['Date'] < '2018-01-01')
val = df.loc[mask]
val.head()

,Burrito,Date,Yelp,Google,Cost,Hunger,Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Beef,Pico,Guac,Cheese,Fries,Sour cream,Great
303,Other,2017-01-07,3.887356,4.167816,8.50,3.9,21.0,21.0,0.74,3.0,4.5,4.1,3.0,3.7,4.0,4,4.2,5.0,0,0,0,0,0,0,False
304,Other,2017-01-07,3.887356,4.167816,7.90,4.0,20.5,21.0,0.72,3.5,4.0,4.0,3.0,4.0,4.5,4,3.8,4.8,0,0,0,0,0,0,False
305,Other,2017-01-10,3.887356,4.167816,4.99,3.5,18.5,22.5,0.75,2.5,4.5,3.0,2.5,3.0,3.0,2,2.0,4.0,0,0,0,0,0,0,False
306,Other,2017-01-12,3.500000,4.500000,7.29,3.5,19.0,21.5,0.70,3.0,2.0,3.5,3.5,3.0,2.5,3,3.2,4.2,0,0,0,0,0,0,False
307,Surf & Turf,2017-01-12,3.887356,4.167816,7.89,3.0,18.5,25.5,0.96,4.0,5.0,4.5,4.0,4.5,3.5,3,4.5,2.5,0,0,0,0,0,0,True


In [35]:
mask = (df['Date'] > '2017-12-31')
test = df.loc[mask]
test.head()

,Burrito,Date,Yelp,Google,Cost,Hunger,Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Beef,Pico,Guac,Cheese,Fries,Sour cream,Great
77,California,2026-04-25,3.887356,4.167816,8.00,4.0,21.590000,22.135765,0.786477,4.5,5.0,5.0,5.0,4.5,5.0,3,5.0,5.0,1,1,1,1,1,0,True
386,California,2018-01-02,3.887356,4.167816,7.25,4.0,20.038233,22.135765,0.786477,4.0,5.0,4.0,5.0,5.0,3.0,3,4.0,5.0,0,0,0,0,0,0,False
387,Other,2018-01-09,4.500000,3.800000,4.19,3.0,20.038233,22.135765,0.786477,3.0,5.0,2.0,2.0,4.0,1.0,4,3.0,4.0,0,0,0,1,0,1,False
388,California,2018-01-12,3.500000,4.300000,7.00,5.0,20.038233,22.135765,0.786477,5.0,5.0,5.0,5.0,5.0,5.0,4,5.0,5.0,0,0,0,0,0,0,True
389,Other,2018-01-12,3.887356,4.167816,8.50,4.0,21.000000,23.500000,0.920000,4.0,4.0,3.0,3.5,1.0,2.0,3,3.0,1.0,0,0,0,0,0,0,False


In [0]:
#Begin with baselines for classification.

In [37]:
target = 'Great'
y_train = train[target]
y_train.value_counts(normalize=True)

False    0.606061
True     0.393939
Name: Great, dtype: float64

In [0]:
majority_class = y_train.mode()[0]
y_pred = [majority_class] * len(y_train)


In [39]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_pred)

0.6060606060606061

In [0]:
#Use scikit-learn for logistic regression.

In [41]:
df.select_dtypes(exclude='number').describe().T


,count,unique,top,freq,first,last
Burrito,373,5,California,154,NaT,NaT
Date,373,155,2016-08-30 00:00:00,26,2016-01-18,2026-04-25
Great,373,2,False,216,NaT,NaT


In [42]:
df.head()

,Burrito,Date,Yelp,Google,Cost,Hunger,Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Beef,Pico,Guac,Cheese,Fries,Sour cream,Great
0,California,2016-01-18,3.500000,4.200000,6.49,3.0,20.038233,22.135765,0.786477,3.0,5.0,3.0,3.5,4.0,4.0,4,4.0,4.0,1,1,1,1,1,0,False
1,California,2016-01-24,3.500000,3.300000,5.45,3.5,20.038233,22.135765,0.786477,2.0,3.5,2.5,2.5,2.0,4.0,3,2.5,5.0,1,1,1,1,1,0,False
2,Carnitas,2016-01-24,3.887356,4.167816,4.85,1.5,20.038233,22.135765,0.786477,3.0,2.0,2.5,3.0,4.5,4.0,3,3.0,5.0,0,1,1,0,0,0,False
3,Asada,2016-01-24,3.887356,4.167816,5.25,2.0,20.038233,22.135765,0.786477,3.0,2.0,3.5,3.0,4.0,5.0,4,4.0,5.0,1,1,1,0,0,0,False
4,California,2016-01-27,4.000000,3.800000,6.59,4.0,20.038233,22.135765,0.786477,4.0,5.0,4.0,3.5,4.5,5.0,2,4.5,4.0,1,1,0,1,1,0,True


In [0]:
target = 'Great'
high_cardinality = ['Date']
features = train.columns.drop([target] + high_cardinality)

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]

In [46]:
import category_encoders as ce

encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_val = encoder.transform(X_val)

ValueError: ignored

In [0]:
X_train

In [57]:
# 1. Import estimator class
from sklearn.linear_model import LogisticRegression
import category_encoders as ce 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# 2. Instantiate this class
log_reg = LogisticRegression(random_state=42)

# 3. Arrange X feature matrices (already did y target vectors)
target = 'Great'
features = [ 'Cost', 'Hunger','Circum', 'Length', 'Volume', 'Tortilla',	
            'Temp', 	'Meat', 	'Fillings', 'Meat:filling',	'Uniformity', 'Salsa',	'Synergy',	'Wrap', 
            'Beef', 	'Pico', 	'Guac', 	'Cheese', 	'Fries', 'Yelp', 'Google', 	'Sour cream']
X_train = train[features]
y_train = train[target]

X_val = val[features]
y_val = val[target]

#scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

#4. Fit the model
log_reg.fit(X_train_scaled, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [58]:
print('Validation Accuracy', log_reg.score(X_val, y_val))

Validation Accuracy 0.44
